## Exercício sobre pytorch

Utilize o dataset 'datasetCarros.csv'.<br>
Usando Pytorch, construa uma rede neural para prever a feature 'PrecoVenda'.<br>

Use uma rede neural feed forward com duas camadas escondidas, com 20 neurônios cada.<br>
Use o critério de perda MSELoss, otimizador Adam e learning rate = 0.001. Considere 1000 épocas.

In [110]:
import pandas as pd
from sklearn.model_selection import train_test_split
from torch import nn
import torch
import numpy as np


# variavel binaria para decidir se vai tentar prever nota ou arrecadação
NOTA = False

class Feedforward(torch.nn.Module):
    
        def __init__(self, input_size, hidden_size):
            super(Feedforward, self).__init__()
            
            self.input_size = input_size
            self.hidden_size  = hidden_size
            self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
            self.fc2 = torch.nn.Linear(self.hidden_size, self.hidden_size)
            self.fc3 = torch.nn.Linear(self.hidden_size, 1)
            
            self.relu = torch.nn.ReLU()
            
            
        def forward(self, x):
            output = self.fc1(x)
            output = self.relu(output)
            
            output = self.fc2(output)
            output = self.relu(output)
            
            output = self.fc3(output)

            return output

device = "cuda" if torch.cuda.is_available() else "cpu"

imdb = pd.read_csv('imdb_top_1000.csv')


imdb = imdb.drop(["Released_Year","Poster_Link","Overview","No_of_Votes","Meta_score","Runtime","Certificate","Genre","Director"],axis=1)
if NOTA:
    imdb = imdb.drop(["Gross"],axis=1)
else:
    imdb= imdb.drop(["IMDB_Rating"],axis=1)
    imdb = imdb[imdb['Gross'].notna()]
    imdb = imdb.reset_index(drop=True)
    imdb["Gross"] = imdb["Gross"].apply(lambda x:  int(str(x).replace(",","")))

transformar atores para colunas categoricas, tirando a redundancia q teria com get_dummies: star1_Al_pacino, star2_Al_pacino

In [111]:
lista_total = []
for _,linha in imdb.iterrows():
    lista_total.append((linha[["Star1","Star2","Star3","Star4"]].values))
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
astros = pd.DataFrame(mlb.fit_transform(lista_total),columns=mlb.classes_)
imdb = pd.concat([imdb,astros],axis=1)
imdb = imdb.drop(["Star1","Star2","Star3","Star4"],axis=1)

In [112]:
if NOTA:
    y = imdb['IMDB_Rating']
    X = imdb.drop(["Series_Title",'IMDB_Rating'],axis=1)
else:
    y = imdb['Gross']
    X = imdb.drop(["Series_Title",'Gross'],axis=1)

y_tensor = torch.tensor(y)
X_tensor = torch.tensor(X.to_numpy())

X_treino, X_teste, y_treino, y_teste = train_test_split(X_tensor, y_tensor, test_size = 0.10, random_state=5)


X_treino = X_treino.float().to(device)
y_treino = y_treino.float().to(device)


X_teste = X_teste.float().to(device)
y_teste = y_teste.float().to(device)

X_tensor = torch.tensor(X.to_numpy())

In [121]:
model = Feedforward(X_tensor.shape[1], 20).to(device)

criterion = torch.nn.MSELoss()
#criterion = torch.nn.L1Loss()

optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)


model.eval()
y_pred = model(X_teste)
antes_treino = criterion(y_pred, y_teste) 
print('Teste - perda antes do treinamento' , antes_treino.item())

model.train()
epoch = 2_500

for epoch in range(epoch):
    optimizer.zero_grad()
    
    # Passe Forward
    y_pred = model(X_treino)
    
    # Computa a perda
    loss = criterion(y_pred, y_treino)
    if epoch % 500 == 0:
        print('Epoch {}: perda treino: {}'.format(epoch, loss.item()))
    
    # Passe de Backward
    loss.backward()
    optimizer.step()


model.eval()
y_pred = model(X_teste)
after_train = criterion(y_pred, y_teste) 
print('Teste - perda depois do treinamento' , after_train.item())  

/home/dnl/.local/share/virtualenvs/aps-BmxP0JNk/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([84])) that is different to the input size (torch.Size([84, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/dnl/.local/share/virtualenvs/aps-BmxP0JNk/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([747])) that is different to the input size (torch.Size([747, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Teste - perda antes do treinamento 1.8832332081332224e+16
Epoch 0: perda treino: 1.6414947319742464e+16
Epoch 500: perda treino: 1.6346451181305856e+16
Epoch 1000: perda treino: 1.5888977034739712e+16
Epoch 1500: perda treino: 1.4922105808224256e+16
Epoch 2000: perda treino: 1.3672965036048384e+16
Teste - perda depois do treinamento 1.6499103982682112e+16


In [117]:
atores = list(imdb.columns.values)

In [118]:
def pegar_atores(nomes):
    torch_tmp = torch.zeros(X_teste[0].shape)
    indices = []
    for i in nomes:
        indices.append(atores.index(i))
    for i in indices:
        torch_tmp[i] = 1
    return torch_tmp

In [119]:
import random
batman_harry_potter = ["Daniel Radcliffe","Emma Watson","Rupert Grint","Christian Bale","Michael Caine"]
harry_potter  = ["Daniel Radcliffe","Emma Watson","Rupert Grint"]
poderoso_chefao = ["Al Pacino","Robert De Niro"]

aleatorio = random.sample(atores, 4)
batman_harry_potter = pegar_atores(batman_harry_potter)
aleatorio = pegar_atores(aleatorio)
harry_potter = pegar_atores(harry_potter)
poderoso_chefao = pegar_atores(poderoso_chefao)

In [120]:
print(model(batman_harry_potter).item())
print(model(harry_potter).item())
print(model(aleatorio).item())
print(model(poderoso_chefao).item())

27194938.0
18133548.0
18129444.0
9073548.0
